# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

Esses valores podem ser atribuídos na função `main`, não é necessária leitura pelo teclado. Entretanto, o programa deve ser genérico, ou seja, funcionará para qualquer outro valor atribuído às 3 variáveis.

In [1]:
#include <stdio.h>

int main(){
    int s=200, n=5;
    float j=1, parcela = s;
    
    for(int i=0; i<5; i++){
        if(i == 0){
            printf("%.2f\n", parcela);
        } else {
            parcela = parcela * (1 + j / 100);
            printf("%.2f\n", parcela);
        }
    }
    
    return 0;
}

200.00
202.00
204.02
206.06
208.12


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de cada parcela X do empréstimo.

Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa em que está inserida. Considere que esta função poderia ser eventualmente reusada em outro programa e deveria ser possível fazê-lo sem que ela dependa de nada do programa em que irá se inserir.

A função deve apenas calcular uma única parcela em cada chamada, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [6]:
#include <stdio.h>

void proximaParcela(int parcelaInicial, int numeroDeParcelas, float juros){
    float parcela = parcelaInicial;
    for(int i=1; i<numeroDeParcelas; i++){
        parcela = parcela * (1 + juros / 100);
    }
    printf("%.2f\n", parcela);
}

int main(){
    // dados iniciais
    int s=200, j=1;

    // chamadas da função
    proximaParcela(s, 1, j);
    proximaParcela(s, 2, j);
    proximaParcela(s, 3, j);
    proximaParcela(s, 4, j);
    proximaParcela(s, 5, j);
    
    return 0;
}

200.00
202.00
204.02
206.06
208.12


## Exercício Parte 3 - Minimizando os Parâmetros

Na solução anterior, você deve ter usado vários parâmetros para a comunicação entre o `main` com a função `proximaParcela`, evitando usar variáveis globais. Modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar todos os valores que estejam relacionados a um empréstimo.

In [4]:
#include <stdio.h>

typedef struct{
    float parcelaAtual;
    float juros;
} Emprestimo;

// cria e retorna uma estrutura de empréstimo
Emprestimo criarEmprestimo(float parcelaInicial, float juros){
    Emprestimo novo;
    novo.parcelaAtual = parcelaInicial;
    novo.juros = juros;
    return novo;
}

// calcula a proxima parcela de um empréstimo dado e atualiza sua estrutura
void proximaParcela(Emprestimo *emprestimo){
    emprestimo->parcelaAtual = emprestimo->parcelaAtual * (1 + emprestimo->juros / 100);
    printf("%.2f\n", emprestimo->parcelaAtual);
}

int main(){
    // dados iniciais
    Emprestimo emprestimo = criarEmprestimo(200, 1);

    // chamadas da função
    proximaParcela(&emprestimo);
    proximaParcela(&emprestimo);
    proximaParcela(&emprestimo);
    proximaParcela(&emprestimo);

    return 0;
}

202.00
204.02
206.06
208.12


## Exercício Parte 4 - Mantendo o Estado

A função que você codificou na solução anterior provavelmente não é capaz de manter o controle do que chamaremos de **estado do empréstimo**, ou seja, dados de qual a parcela corrente ou da última parcela calculada.

Modifique a função (módulo) `proximaParcela` acima de modo que ela seja capaz de guardar o **estado do empréstimo** por sua conta, sem que o `main` precise ficar informando este estado por parâmetros (por exemplo). Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência das funções com o programa em que ela está inserida.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente. O ideal é que o main não tenha acesso ao modo como as funções vão representar o **estado do empréstimo**, para que toda a lógica do empréstimo fique dentro das funções.

In [37]:
#include <stdio.h>

typedef struct{
    float parcelaInicial;
    float parcelaAtual;
    int numeroDeParcelas;
    int numeroParcelasPagas;
    float juros;
} Emprestimo;

// cria e retorna uma estrutura de empréstimo
Emprestimo criarEmprestimo(float parcelaInicial, int numeroDeParcelas, float juros){
    Emprestimo novo;
    novo.parcelaInicial = parcelaInicial;
    novo.parcelaAtual = parcelaInicial;
    novo.numeroDeParcelas = numeroDeParcelas;
    novo.numeroParcelasPagas = 0;
    novo.juros = juros;
    return novo;
}

// calcula a proxima parcela de um empréstimo dado e atualiza sua estrutura
void proximaParcela(Emprestimo *emprestimo){
    if(emprestimo->numeroParcelasPagas == 0){
        printf("%.2f\n", emprestimo->parcelaInicial);
        emprestimo->numeroParcelasPagas += 1;
        return;
    }
    emprestimo->parcelaAtual = emprestimo->parcelaAtual * (1 + emprestimo->juros / 100);
    if(emprestimo->numeroParcelasPagas == emprestimo->numeroDeParcelas){
        printf("Todas as parcelas foram pagas!\n");
        return;
    }
    emprestimo->numeroParcelasPagas += 1;
    printf("%.2f\n", emprestimo->parcelaAtual);
}

int main(){
    // dados iniciais
    Emprestimo emprestimo = criarEmprestimo(200, 5, 1);

    // chamadas da função
    proximaParcela(&emprestimo);
    proximaParcela(&emprestimo);
    proximaParcela(&emprestimo);
    proximaParcela(&emprestimo);
    proximaParcela(&emprestimo);
    proximaParcela(&emprestimo);

    return 0;
}

200.00
202.00
204.02
206.06
208.12
Todas as parcelas foram pagas!


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada **estado de empréstimo** separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, as funções de cálculo do empréstimo ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que comporte até 100 empréstimo e, cada novo empréstimo, só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo (até o limite de 100).

O seu programa deve apresentar o exemplo acima como teste.

In [38]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

typedef struct{
    char nome[15];
    float parcelaInicial;
    float parcelaAtual;
    int numeroDeParcelas;
    int numeroParcelasPagas;
    float juros;
} Emprestimo;

// cria uma estrutura de empréstimo e a adiciona a uma lista dada
void criarEmprestimo(Emprestimo lista[100], char nome[15], float parcelaInicial, int numeroDeParcelas, float juros, int *maximoParcelas, int *nEmprestimos){
    Emprestimo novo;
    strcpy(novo.nome, nome);
    novo.parcelaInicial = parcelaInicial;
    novo.parcelaAtual = parcelaInicial;
    novo.numeroDeParcelas = numeroDeParcelas;
    novo.numeroParcelasPagas = 0;
    novo.juros = juros;

    lista[*nEmprestimos] = novo;

    if(numeroDeParcelas > *maximoParcelas){
        *maximoParcelas = numeroDeParcelas;
    }
    *nEmprestimos += 1;
}

// calcula a proxima parcela de um empréstimo dado e atualiza sua estrutura
void proximaParcela(Emprestimo *emprestimo){
    if(emprestimo->numeroParcelasPagas == 0){
        emprestimo->numeroParcelasPagas += 1;
        printf("%s: parcela 1 é %.2f\n", emprestimo->nome, emprestimo->parcelaInicial);
        return;
    }
    emprestimo->parcelaAtual = emprestimo->parcelaAtual * (1 + emprestimo->juros / 100);
    emprestimo->numeroParcelasPagas += 1;
    printf("%s: parcela %d é %.2f\n", emprestimo->nome, emprestimo->numeroParcelasPagas, emprestimo->parcelaAtual);
}

// calcula e imprime todas as parcelas de uma lista de empréstimos
void calculaParcelas(Emprestimo lista[100], int nEmprestimos, int maximoParcelas){
    for(int i=0; i < maximoParcelas; i++){
        for(int j=0; j < nEmprestimos; j++){
            if(lista[j].numeroParcelasPagas < lista[j].numeroDeParcelas){
                proximaParcela(&lista[j]);
            }
        }
    }
}

int main(){
    // dados iniciais
    Emprestimo emprestimos[100];
    int maximoParcelas = 0, nEmprestimos = 0;

    criarEmprestimo(emprestimos, "Emprestimo 1", 200, 5, 1, &maximoParcelas, &nEmprestimos);
    criarEmprestimo(emprestimos, "Emprestimo 2", 500, 7, 2, &maximoParcelas, &nEmprestimos);

    // chamadas da função
    calculaParcelas(emprestimos, nEmprestimos, maximoParcelas);

    return 0;
}

Emprestimo 1: parcela 1 é 200.00
Emprestimo 2: parcela 1 é 500.00
Emprestimo 1: parcela 2 é 202.00
Emprestimo 2: parcela 2 é 510.00
Emprestimo 1: parcela 3 é 204.02
Emprestimo 2: parcela 3 é 520.20
Emprestimo 1: parcela 4 é 206.06
Emprestimo 2: parcela 4 é 530.60
Emprestimo 1: parcela 5 é 208.12
Emprestimo 2: parcela 5 é 541.22
Emprestimo 2: parcela 6 é 552.04
Emprestimo 2: parcela 7 é 563.08
